# SA-CCR Euler allocation of exemplary equity portfolio

We set up a collateral agreements with no thresholds or MTA and add two equity options to it.
A big put option on Adidas and a smaller call option on Adidas.

We consider three cases of margining

1. No margining
2. VM only
3. VM and bilateral IM

In [1]:
import QuantLib as ql
from allocation.eulerAllocator import EulerAllocator
from collateralAgreement.collateralAgreement import CollateralAgreement, Margining, InitialMargining
from instruments.equity_instruments.equityOption import EquityOption
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import Stock, TradeType, TradeDirection
from marketdata import init_marketdata
asdf =1

In [2]:
ca = CollateralAgreement(threshold=0,
                         threshold_vm=0,
                         mta=0,
                         margining=Margining.UNMARGINED,
                         initialMargining=InitialMargining.NO_IM)
ca.link_sa_ccr_instance(SA_CCR(ca))

ca_vm = CollateralAgreement(threshold=0,
                            threshold_vm=0,
                            mta=0,
                            margining=Margining.MARGINED,
                            initialMargining=InitialMargining.NO_IM)
ca_vm.link_sa_ccr_instance(SA_CCR(ca_vm))

ca_im = CollateralAgreement(threshold=0,
                            threshold_vm=0,
                            mta=0,
                            margining=Margining.MARGINED,
                            initialMargining=InitialMargining.SIMM)
ca_im.link_sa_ccr_instance(SA_CCR(ca_im))

In [3]:
eq_opt_ads_call = EquityOption(underlying=Stock.ADS,
                               maturity=ql.Period(1, ql.Years),
                               notional=2000000,
                               tradeType=TradeType.CALL,
                               tradeDirection=TradeDirection.LONG)

eq_opt_ads_put = EquityOption(underlying=Stock.ADS,
                              maturity=ql.Period(1, ql.Years),
                              notional=3000000,
                              tradeType=TradeType.PUT,
                              tradeDirection=TradeDirection.LONG)

When putting these two trades in the portfolio and allocating them we yield the following:

In [4]:
ca.add_trades([eq_opt_ads_call, eq_opt_ads_put])
ca_vm.add_trades([eq_opt_ads_call, eq_opt_ads_put])
ca_im.add_trades([eq_opt_ads_call,eq_opt_ads_put])

In [5]:
eulerAllocator = EulerAllocator(ca)
allocation_no_margin = eulerAllocator.allocate_ead()
eulerAllocator.ca=ca_vm
allocation_with_vm = eulerAllocator.allocate_ead()
eulerAllocator.ca = ca_im
allocation_with_im_and_im = eulerAllocator.allocate_ead()
allocation_im = eulerAllocator.allocate_im()